In [4]:

from coinor.blimpy import Queue, PriorityQueue
import numpy as np
import heapq
from math import sqrt, pow
class Event(object):
    '''
    Basically there are two event types, arrival and departure, this class can
    generate instances of arrival and departure events. See the file
    documentation for description of attributes.
    '''
    def __init__(self, eventType, eventNumber, eventTime, serverNumber = None,que_index=0):
        '''
        Constructor of the class. 
        parameters:
        eventType: arrival and depart
        eventNumber 
        eventTime
        serverNumber:the index of a worker that deals with the parcel
        que_index=0:0=regular parcel,1=priority parcel
        
        '''
        self.que_index=que_index
        self.number = eventNumber
        self.eventType = eventType
        self.eventTime = eventTime
        self.serverNumber = serverNumber
        if self.eventType == ARRIVE:
            self.name = 'arrival'+str(self.number)
        elif self.eventType == DEPART:
            self.name = 'departure'+str(self.number)
        else:
            print("unknown event type")

    def __gt__(self, other):
        '''
        Returns True if self.eventTime > other.eventTime
        '''
        return self.eventTime > other.eventTime

    def __ge__(self, other):
        '''
        Returns True if self.eventTime >= other.eventTime
        '''
        return self.eventTime >= other.eventTime

    def __lt__(self, other):
        '''
        Returns True if self.eventTime < other.eventTime
        '''
        return self.eventTime < other.eventTime

    def __le__(self, other):
        '''
        Returns True if self.eventTime <= other.eventTime
        '''
        return self.eventTime <= other.eventTime

    def __eq__(self, other):
        '''
        returns True if self.eventTime == other.eventTime
        '''
        return self.eventTime == other.eventTime

    def __ne__(self, other):
        '''
        Returns True if self.eventTime != other.eventTime
        '''
        return self.eventTime != other.eventTime

class Customer(object):
    '''
    Customer class. A basic class with only constructor method and 3 到达时间
    attributes.
    '''
    def  __init__(self, entryTime, serviceTime, customerNumber):
        '''
        initial all values of parcels 
        paramaters:entryTime
                   serviceTime
                   customernumber
       
        '''
        self.entryTime = entryTime
        self.serviceTime = serviceTime
        self.number = customerNumber
 
class EventQueue(object):
    '''
    This class holds the attributes and methods for the simulation of M/M/s
    queueing system
    '''
    def __init__(self, seedInput = 111, IAT = 4, ST = 2, server_num = 9):
        '''
        Constructor of the class, sets initial values for class attributes
        paramaters:seedinput(int)
                   IAT(int): the package according with the poisson process lam=4
                   ST(int):the assumption time of each worker to deal with one parcel 
                   server_num:the number of post office workers
        '''
        self.busytime=0
        self.freetime=0        
        np.random.seed=seedInput
        self.seed=seedInput
        self.IAT = IAT
        self.ST = ST
        self.currentTime = 0.0
        self.eventTable = []

        self.waitingTime = {}
        # time in system for each parcel
        self.TIS = {}
        # package time for the corresponding parcel
        self.serviceTime = {}
        self.eventCounter = 0
        self.customerCounter = 0
        self.queue_num = 2
        self.server_num=server_num
        self.server = [IDLE for i in range(self.server_num)]        
        self.sqList = [Queue() for i in range(self.queue_num)]
        self.add_event(ARRIVE, self.currentTime)         
        self.requ=0
        self.pri=0
    def process(self,event): 
        '''
        process the arrival and departure event
        '''
        
        if event.eventType == ARRIVE:
            #### if the event is arrival check the parcel whether is belongs to priority or regular parcel,
            #### que_index=1 is priority parcel ,que index=0 is regular parcel
            
            serviceTime = np.random.exponential(self.ST)            
            if event.que_index==1:
                if self.sqList[1].isEmpty():
                    ##chcek priority queue is empty, if it is empty##
                    index=self.change_state()                                       
                    if index is None:
                        ### check if there is free worker if there's no free worker 
                        #add the parcel to priority queue
                        
                        self.busytime+=1
                        parcel=Customer(self.currentTime, serviceTime, self.customerCounter)
                        self.sqList[1].enqueue(parcel)
                        self.pri+=1                        
                    else: 
                        #### arrange worker to pack parcel and record the package time and the index of the worker
                        self.waitingTime[self.customerCounter] = 0
                        self.serviceTime[self.customerCounter] = serviceTime
                        self.add_event(DEPART, self.currentTime + serviceTime,index,event.que_index)
                else:
                    ### if the priority is not empty and the parcel to priority queue
                    self.busytime+=1
                    parcel=Customer(self.currentTime, serviceTime, self.customerCounter)
                    self.sqList[1].enqueue(parcel)
                    self.pri+=1                    
            elif event.que_index==0:
                #### if the parcel is regular parcel###
                serviceTime = np.random.exponential(self.ST)
                ### check the priority queue is empty if it is empty then check the regular queue if it is empty#
                if self.sqList[1].isEmpty():
                    if self.sqList[0].isEmpty():
                        index=self.change_state()                        
                        if index is None:##if regular and priority queue is empty,check if there is free worker
                            self.busytime+=1 ### if there is no free worker add the parcel to regular queue
                            parcel=Customer(self.currentTime, serviceTime, self.customerCounter)
                            self.sqList[0].enqueue(parcel)
                            self.requ+=1
                        else: ##if there is free worker record the index of the worker and record the package time
                            self.waitingTime[self.customerCounter] = 0
                            self.serviceTime[self.customerCounter] = serviceTime
                            self.add_event(DEPART, self.currentTime + serviceTime,index,event.que_index)
                    else:###if the regular queue is not empty add the parcel to the regular queue
                        self.busytime+=1
                        parcel=Customer(self.currentTime, serviceTime, self.customerCounter)
                        self.sqList[0].enqueue(parcel)
                        self.requ+=1                        
                else:### if the priority is not empty ,add the parcel to the regular queue
                    self.busytime+=1
                    parcel=Customer(self.currentTime, serviceTime, self.customerCounter)
                    self.sqList[0].enqueue(parcel)
                    self.requ+=1                    
            self.customerCounter += 1  ## add the number of parcel
            rand=np.random.binomial(1,0.1)
            self.add_event(ARRIVE, self.currentTime+np.random.exponential(1/self.IAT),que_index=rand) ##add a new arrival event and it is a possion process
        elif event.eventType == DEPART:   ## if the event is departure         
            if event.que_index==1:        ## check what the type of the pacel departs
                q = self.sqList[event.que_index]                
                if not q.isEmpty():   ## if the priority is not empty and minus 1 of the priority and recode the waiting time of package
                    self.pri-=1
                    parcel=q.dequeue()
                    self.waitingTime[parcel.number] = (self.currentTime - 
                                                     parcel.entryTime)   
                    self.serviceTime[parcel.number] = parcel.serviceTime   
                    self.add_event(DEPART, self.currentTime + parcel.serviceTime,
                               event.serverNumber,event.que_index)
                    self.server[event.serverNumber] = BUSY
                elif q.isEmpty():  ## if the priority is  empty then check the regular queue
                    regular=self.sqList[0]
                    if regular.isEmpty():                        
                        self.server[event.serverNumber] = IDLE
                    else:
                        parcel=regular.dequeue()
                        self.requ-=1
                        self.waitingTime[parcel.number] = (self.currentTime - 
                                                     parcel.entryTime)   
                        self.serviceTime[parcel.number] = parcel.serviceTime  
                        self.add_event(DEPART, self.currentTime + parcel.serviceTime,
                               event.serverNumber,0)
                        self.server[event.serverNumber] = BUSY
            elif event.que_index==0:   
                ### if the regular queue is empty then check priority queue
                ### if the regular queue is not empty then chcek priority is empty
                ## if the priority is not empty ,deal with priority queue first
                q = self.sqList[0] 
                if q.isEmpty():
                    if self.sqList[1].isEmpty():                        
                        self.server[event.serverNumber] = IDLE
                    else:
                        parcel=self.sqList[1].dequeue()
                        self.pri-=1
                        self.waitingTime[parcel.number] = (self.currentTime - 
                                                     parcel.entryTime)   
                        self.serviceTime[parcel.number] = parcel.serviceTime   
                        self.add_event(DEPART, self.currentTime + parcel.serviceTime,
                               event.serverNumber,1)
                        self.server[event.serverNumber] = BUSY                    
                if not q.isEmpty():
                    if self.sqList[1].isEmpty():
                        parcel=q.dequeue()
                        self.requ-=1
                        self.waitingTime[parcel.number] = (self.currentTime - 
                                                     parcel.entryTime)  
                        self.serviceTime[parcel.number] = parcel.serviceTime  
                        self.add_event(DEPART, self.currentTime + parcel.serviceTime,
                               event.serverNumber,event.que_index)
                        self.server[event.serverNumber] = BUSY                        
                    else:
                        parcel=self.sqList[1].dequeue()
                        self.pri-=1
                        self.waitingTime[parcel.number] = (self.currentTime - 
                                                     parcel.entryTime) 
                        self.serviceTime[parcel.number] = parcel.serviceTime   
                        self.add_event(DEPART, self.currentTime + parcel.serviceTime,
                               event.serverNumber,1)
                        self.server[event.serverNumber] = BUSY
         
    def change_state(self):
        '''
        a list containing workers 
        if the worker is free the state is idle
        if the worker is not free the state is busy
        '''
        for i in range(len(self.server)):
            if self.server[i]==IDLE:
                self.server[i]=BUSY
                return i                        
    def print_stat(self):
        '''
        caculate the statistic such as waiting time ,busy proportion
        '''
        for k in self.waitingTime:
            self.TIS[k] = self.waitingTime[k] + self.serviceTime[k]
        n1 = len(self.waitingTime)
        n2 = len(self.serviceTime)
        n3 = len(self.TIS)
        av1 = sum(self.waitingTime.values())/n1
        av2 = sum(self.serviceTime.values())/n2
        av3 = sum(self.TIS.values())/n3        
        stdev1 = sqrt(sum([pow(self.waitingTime[i]-av1,2) for i in self.waitingTime])/(n1-1))
        stdev2 = sqrt(sum([pow(self.serviceTime[i]-av2,2) for i in self.serviceTime])/(n2-1))
        stdev3 = sqrt(sum([pow(self.TIS[i]-av3,2) for i in self.TIS])/(n3-1))
        ic_av1=(av1-1.96*stdev1,av1+1.96*stdev1)
        print ('\n')
        print ('Seed: ', self.seed)
        print ('Simulation ended at ', self.currentTime)
        print ('=========================\t STATISTICS',)
        print (' \t =========================')
        print ('\t\t\tObservations\tAverage\t\tStDev')
        print ('Waiting Time\t\t', n1, '\t\t', av1, '\t', stdev1)
        print ('Service Time\t\t', n2, '\t\t', av2, '\t', stdev2)
        print ('Time is System\t\t', n3, '\t\t', av3, '\t', stdev3)
        print (ic_av1)
        print(self.busytime)
        busy_pro=self.busytime/n1
        return (busy_pro,av1),(av2,av3)
    def add_event(self, eventType, eventTime, serverNumber = None,que_index=0):
        '''
        add the event to the eventtable according to the happening time of the event
        '''        
        self.eventCounter += 1
        data=Event(eventType, self.eventCounter, eventTime, serverNumber,que_index)  
        heapq.heappush(self.eventTable,data)
    def get_event(self):
        '''
        get the first event in the eventtable
        '''                
        obj=heapq.heappop(eq.eventTable)        
        self.currentTime = obj.eventTime
        return obj
    def simulate(self, simulationLength):
        '''
        until the current time >simulationlength,  the system will stop
        '''       
        while self.currentTime < simulationLength:
            event = self.get_event()            
            self.process(event)
            

           

In [5]:
ARRIVE=0
DEPART=1
IDLE=0
BUSY=1
list1=[]
list2=[]
list3=[]
list4=[]
for i in range(50):
    eq = EventQueue(seedInput = 1111, IAT =4, ST = 2,server_num =9)
    eq.simulate(2000)
    
    
    x,y=eq.print_stat()
    busy_pro,av1=x
    av2,av3=y
    
    list1.append(busy_pro)
    list2.append(av1)
    list3.append(av2)
    list4.append(av3)


    




Seed:  1111
Simulation ended at  2000.2314864984157
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 7990 		 1.4454424561121177 	 2.228124892701263
Service Time		 7990 		 1.996932464365664 	 1.993962536830852
Time is System		 7990 		 3.4423749204777727 	 2.992708171188128
(-2.921682333582358, 5.812567245806593)
5286


Seed:  1111
Simulation ended at  2000.1256009964113
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 8074 		 1.1145301547648139 	 1.6523195203019723
Service Time		 8074 		 2.000572808534421 	 1.9692509734380015
Time is System		 8074 		 3.1151029632992264 	 2.5470409691131475
(-2.124016105027052, 4.353076414556679)
5332


Seed:  1111
Simulation ended at  2000.0684803748245
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 7972 		 0.763951105019107 	 1.1000373482920542
Service Time		 7972 		 1.963577459115971 	 2.02857367687117
Time is System		 7972 		 2.727528564135075



Seed:  1111
Simulation ended at  2000.0222166137207
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 7900 		 0.9387377153812054 	 1.4069850078684258
Service Time		 7900 		 1.9850545954042755 	 1.9877097594196063
Time is System		 7900 		 2.9237923107854717 	 2.426528441120803
(-1.8189529000409093, 3.6964283308033203)
4723


Seed:  1111
Simulation ended at  2000.445243585569
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 7858 		 0.8176309390812899 	 1.202847108930399
Service Time		 7858 		 1.9941975507402323 	 2.0069105987724716
Time is System		 7858 		 2.8118284898215227 	 2.327717045525938
(-1.5399493944222917, 3.1752112725848716)
4543


Seed:  1111
Simulation ended at  2000.1417332403905
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 8123 		 2.011480913958466 	 2.896262778229069
Service Time		 8123 		 2.0289588185884018 	 2.0877589010641264
Time is System		 8123 		 4.04043973



Seed:  1111
Simulation ended at  2000.036747555858
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 8084 		 1.0996753449176124 	 1.543133640127948
Service Time		 8084 		 2.0201395268993894 	 2.0131685700156465
Time is System		 8084 		 3.1198148718170122 	 2.5384488767543862
(-1.9248665897331658, 4.1242172795683905)
5559


Seed:  1111
Simulation ended at  2000.1995569506867
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 7764 		 0.870249304387899 	 1.3266487731597956
Service Time		 7764 		 2.0358974472726294 	 2.020752226609224
Time is System		 7764 		 2.906146751660515 	 2.411029085940214
(-1.7299822910053, 3.4704808997810983)
4715


Seed:  1111
Simulation ended at  2000.0360326996995
=========================	 STATISTICS
			Observations	Average		StDev
Waiting Time		 7910 		 1.1940823579629916 	 1.701454342607933
Service Time		 7910 		 2.042328367922119 	 2.021466051404482
Time is System		 7910 		 3.23641072588511

In [57]:
print(np.mean(list1))
print(np.mean(list2))
print(np.mean(list3))
print(np.mean(list4))

0.6432934876970999
1.3276137326670254
1.9961681896881467
3.323781922355171


In [50]:
busy=np.mean(list1)-1.96*np.std(list1),np.mean(list1)+1.96*np.std(list1)
wait=np.mean(list2)-1.96*np.std(list2),np.mean(list2)+1.96*np.std(list2)
service=np.mean(list3)-1.96*np.std(list3),np.mean(list3)+1.96*np.std(list3)
total=np.mean(list4)-1.96*np.std(list4),np.mean(list4)+1.96*np.std(list4)
    
    
        

In [56]:
print("95% CI average busy proportion"+str(busy))
print("95% CI average waiting time"+str(wait))
print("95% CI average service time"+str(service))
print("95% CI average time in system"+str(total))

95% CI average busy proportion(0.5739000685242033, 0.7126869068699966)
95% CI average waiting time(0.655808501042896, 1.9994189642911548)
95% CI average service time(1.9588970313512637, 2.0334393480250297)
95% CI average time in system(2.641594918028102, 4.00596892668224)
